# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 25 2022 2:21PM,242678,10,8321686,"ClearSpec, LLC",Released
1,Jul 25 2022 2:21PM,242678,10,8321701,"ClearSpec, LLC",Released
2,Jul 25 2022 2:21PM,242678,10,8321702,"ClearSpec, LLC",Released
3,Jul 25 2022 2:17PM,242679,10,CLI00341.1,"CLINUVEL, Inc.",Released
4,Jul 25 2022 2:17PM,242679,10,CLI00342.1,"CLINUVEL, Inc.",Released
5,Jul 25 2022 2:00PM,242675,10,8321597,Yusen – 3D Matrix,Released
6,Jul 25 2022 1:59PM,242674,10,8321685,"Snap Medical Industries, LLC",Released
7,Jul 25 2022 1:59PM,242674,10,SNAP30737772,"Snap Medical Industries, LLC",Released
8,Jul 25 2022 1:59PM,242674,10,SNAP30752688,"Snap Medical Industries, LLC",Released
9,Jul 25 2022 1:59PM,242674,10,SNAP30759156,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,242672,Released,1
33,242674,Released,6
34,242675,Released,1
35,242678,Released,3
36,242679,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242672,NaN,NaN,1.0
242674,NaN,NaN,6.0
242675,NaN,NaN,1.0
242678,NaN,NaN,3.0
242679,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242576,0.0,0.0,1.0
242579,0.0,0.0,1.0
242581,0.0,0.0,1.0
242583,0.0,0.0,1.0
242611,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242576,0,0,1
242579,0,0,1
242581,0,0,1
242583,0,0,1
242611,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242576,0,0,1
1,242579,0,0,1
2,242581,0,0,1
3,242583,0,0,1
4,242611,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242576,,,1
1,242579,,,1
2,242581,,,1
3,242583,,,1
4,242611,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 25 2022 2:21PM,242678,10,"ClearSpec, LLC"
3,Jul 25 2022 2:17PM,242679,10,"CLINUVEL, Inc."
5,Jul 25 2022 2:00PM,242675,10,Yusen – 3D Matrix
6,Jul 25 2022 1:59PM,242674,10,"Snap Medical Industries, LLC"
12,Jul 25 2022 1:52PM,242672,10,Eywa Pharma Inc.
13,Jul 25 2022 1:47PM,242671,10,"Citieffe, Inc."
14,Jul 25 2022 1:36PM,242670,10,"Citieffe, Inc."
15,Jul 25 2022 1:18PM,242667,10,"Methapharm, Inc."
21,Jul 25 2022 1:17PM,242666,10,Emerginnova
22,Jul 25 2022 1:15PM,242665,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 25 2022 2:21PM,242678,10,"ClearSpec, LLC",,,3
1,Jul 25 2022 2:17PM,242679,10,"CLINUVEL, Inc.",,,2
2,Jul 25 2022 2:00PM,242675,10,Yusen – 3D Matrix,,,1
3,Jul 25 2022 1:59PM,242674,10,"Snap Medical Industries, LLC",,,6
4,Jul 25 2022 1:52PM,242672,10,Eywa Pharma Inc.,,,1
5,Jul 25 2022 1:47PM,242671,10,"Citieffe, Inc.",,,1
6,Jul 25 2022 1:36PM,242670,10,"Citieffe, Inc.",,,1
7,Jul 25 2022 1:18PM,242667,10,"Methapharm, Inc.",,,6
8,Jul 25 2022 1:17PM,242666,10,Emerginnova,,,1
9,Jul 25 2022 1:15PM,242665,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 25 2022 2:21PM,242678,10,"ClearSpec, LLC",3,,
1,Jul 25 2022 2:17PM,242679,10,"CLINUVEL, Inc.",2,,
2,Jul 25 2022 2:00PM,242675,10,Yusen – 3D Matrix,1,,
3,Jul 25 2022 1:59PM,242674,10,"Snap Medical Industries, LLC",6,,
4,Jul 25 2022 1:52PM,242672,10,Eywa Pharma Inc.,1,,
5,Jul 25 2022 1:47PM,242671,10,"Citieffe, Inc.",1,,
6,Jul 25 2022 1:36PM,242670,10,"Citieffe, Inc.",1,,
7,Jul 25 2022 1:18PM,242667,10,"Methapharm, Inc.",6,,
8,Jul 25 2022 1:17PM,242666,10,Emerginnova,1,,
9,Jul 25 2022 1:15PM,242665,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 25 2022 2:21PM,242678,10,"ClearSpec, LLC",3,,
1,Jul 25 2022 2:17PM,242679,10,"CLINUVEL, Inc.",2,,
2,Jul 25 2022 2:00PM,242675,10,Yusen – 3D Matrix,1,,
3,Jul 25 2022 1:59PM,242674,10,"Snap Medical Industries, LLC",6,,
4,Jul 25 2022 1:52PM,242672,10,Eywa Pharma Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 25 2022 2:21PM,242678,10,"ClearSpec, LLC",3.0,NaN,NaN
1,Jul 25 2022 2:17PM,242679,10,"CLINUVEL, Inc.",2.0,NaN,NaN
2,Jul 25 2022 2:00PM,242675,10,Yusen – 3D Matrix,1.0,NaN,NaN
3,Jul 25 2022 1:59PM,242674,10,"Snap Medical Industries, LLC",6.0,NaN,NaN
4,Jul 25 2022 1:52PM,242672,10,Eywa Pharma Inc.,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 25 2022 2:21PM,242678,10,"ClearSpec, LLC",3.0,0.0,0.0
1,Jul 25 2022 2:17PM,242679,10,"CLINUVEL, Inc.",2.0,0.0,0.0
2,Jul 25 2022 2:00PM,242675,10,Yusen – 3D Matrix,1.0,0.0,0.0
3,Jul 25 2022 1:59PM,242674,10,"Snap Medical Industries, LLC",6.0,0.0,0.0
4,Jul 25 2022 1:52PM,242672,10,Eywa Pharma Inc.,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6066006,104.0,3.0,0.0
15,727905,29.0,57.0,2.0
16,242634,1.0,0.0,0.0
19,242663,18.0,2.0,0.0
21,485282,0.0,2.0,0.0
22,242665,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6066006,104.0,3.0,0.0
1,15,727905,29.0,57.0,2.0
2,16,242634,1.0,0.0,0.0
3,19,242663,18.0,2.0,0.0
4,21,485282,0.0,2.0,0.0
5,22,242665,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,104.0,3.0,0.0
1,15,29.0,57.0,2.0
2,16,1.0,0.0,0.0
3,19,18.0,2.0,0.0
4,21,0.0,2.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,104.0
1,15,Released,29.0
2,16,Released,1.0
3,19,Released,18.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21,22
Status,,,,,,
Completed,0.0,2.0,0.0,0.0,0.0,0.0
Executing,3.0,57.0,0.0,2.0,2.0,0.0
Released,104.0,29.0,1.0,18.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21,22
0,Completed,0.0,2.0,0.0,0.0,0.0,0.0
1,Executing,3.0,57.0,0.0,2.0,2.0,0.0
2,Released,104.0,29.0,1.0,18.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21,22
0,Completed,0.0,2.0,0.0,0.0,0.0,0.0
1,Executing,3.0,57.0,0.0,2.0,2.0,0.0
2,Released,104.0,29.0,1.0,18.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()